# Playground for strokes graph

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import folium
from itertools import combinations
from collections import Counter
import shapely

In [2]:
streets = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")

In [3]:
# Clean data
streets = momepy.remove_false_nodes(streets)
# Add fixed ID to each edge
streets["id"] = streets.index
# Transform into primal graph
G_primal = momepy.gdf_to_nx(streets, approach="primal")
points_primal, lines_primal = momepy.nx_to_gdf(G_primal, points=True, lines=True)
# Use COINS on primal graph edges
coins = momepy.COINS(lines_primal)
# List the stroke for each edge
stroke_attribute = coins.stroke_attribute()
# List each edge for each stroke
stroke_gdf = coins.stroke_gdf()
stroke_gdf["stroke_id"] = stroke_gdf.index
stroke_gdf["edge_ids"] = stroke_gdf.stroke_id.apply(lambda x: list(lines_primal.iloc[stroke_attribute[stroke_attribute == x].index]["id"]))
# Dictionary mapping to each edge ID the edge index
d_lines_primal_id2idx = {val:list(lines_primal[lines_primal["id"] == val].index)[0] for val in lines_primal["id"]}
# Add stroke ID to each edge
nx.set_edge_attributes(G_primal, {e: int(stroke_attribute[d_lines_primal_id2idx[G_primal.edges[e]["id"]]]) for e in G_primal.edges}, "stroke_id")
points_primal, lines_primal = momepy.nx_to_gdf(G_primal, points=True, lines=True)


In [4]:
# m = stroke_gdf.explore(tiles="cartodb-positron", column="stroke_id", cmap="tab10", name="strokes")
# lines_primal.explore(m=m, column="id", name = "lines", cmap="viridis")
# folium.LayerControl().add_to(m)
# m

In [5]:
# m = stroke_gdf.explore(tiles="cartodb-positron", column="stroke_id", cmap="Set2", name="strokes")
# lines.explore(m=m, column="stroke_id", name = "lines", cmap="Set2")
# folium.LayerControl().add_to(m)
# m

In [34]:
G_dual = momepy.gdf_to_nx(lines_primal, approach="dual", angles=True)
d_nodes_dual_id2idx = {G_dual.nodes[n]["id"]:n for n in G_dual.nodes}
d_lines_primal_idx2id = {v:k for k,v in d_lines_primal_id2idx.items()}
lines_primal["angles"] = [[(G_dual.nodes[v]["id"], d["angle"]) for _, v, d in G_dual.edges(d_nodes_dual_id2idx[d_lines_primal_idx2id[idx]], data=True)] for idx in list(lines_primal.index)]

In [62]:
angles_strokes_all = []
for idx in list(stroke_gdf.index):
    array = list(lines_primal[lines_primal["id"].isin(stroke_gdf.iloc[idx]["edge_ids"])]["angles"])
    flat_array = []
    for arr in array:
        for val in arr:
            flat_array.append(val)
    angles_strokes_singular = {}
    for val in flat_array:
        stroke_id = int(lines_primal.iloc[d_lines_primal_id2idx[val[0]]]["stroke_id"])
        if stroke_id == idx:
            pass
        elif stroke_id not in angles_strokes_singular:
            angles_strokes_singular[stroke_id] = [val[1]]
        else:
            angles_strokes_singular[stroke_id].append(val[1])
    angles_strokes_all.append(angles_strokes_singular)
stroke_gdf["angles_with_strokes"] = angles_strokes_all

In [ ]:
G_stroke = nx.Graph()
G_stroke.graph["crs"] = G_primal.graph["crs"]
# Create a node for each stroke with the right features
G_stroke.add_nodes_from([[val, {attr:(list(stroke_gdf[stroke_gdf["stroke_id"] == val][attr])[0] if attr != "geometry" else stroke_gdf[stroke_gdf["stroke_id"] == val].geometry[val]) for attr in list(stroke_gdf)}] for val in list(stroke_gdf["stroke_id"])])
# For all node, put its geometry at the center of the LineString
for n in G_stroke.nodes:
    G_stroke.nodes[n]["geometry"] = stroke_gdf.iloc[n].geometry.interpolate(0.5, normalized=True)
    G_stroke.nodes[n]["x"] = G_stroke.nodes[n]["geometry"].xy[0]
    G_stroke.nodes[n]["y"] = G_stroke.nodes[n]["geometry"].xy[1]
for n in G_stroke.nodes:
    for k in stroke_gdf.iloc[n]["angles_with_strokes"]:
        G_stroke.add_edge(n, k, geometry = shapely.LineString(counting = sum(stroke_gdf.iloc[n]["angles_with_strokes"][k]), angles = stroke_gdf.iloc[n]["angles_with_strokes"][k])

In [68]:
points_stroke, lines_stroke = momepy.nx_to_gdf(G_stroke, points=True, lines=True)
m = stroke_gdf.explore(tiles="cartodb-positron", column="stroke_id", cmap="tab10", name="strokes")
points_primal.explore(m=m, name="points_primal")
lines_primal.explore(m=m, name="lines_primal")
points_stroke.explore(m=m, color="black", marker_kwds={"radius":10}, name="points_stroke")
lines_stroke.explore(m=m, color="blue", name="lines_stroke")
folium.LayerControl().add_to(m)
m

/var/folders/mb/_ysy1pzs13qgnh9b942_7lkh0000gn/T/ipykernel_8655/1971550804.py:1: UserWarning: Approach is not set. Defaulting to 'primal'.
  points_stroke, lines_stroke = momepy.nx_to_gdf(G_stroke, points=True, lines=True)


ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Use GeoDataFrame.set_geometry to set the active geometry column.